# Machine Learning
## Programming Assignment 2: KNN

#### Instructions:
The aim of this assignment is to give you hands-on experience with a real-life machine learning application.
You will be analyzing the sentiment of tweets using KNN classification.
You can only use the Python programming language and Jupyter Notebooks.
Please use procedural programming style and comment your code thoroughly.
There are two parts of this assignment. In part 1, you can use NumPy, Pandas, Matplotlib, and any other standard Python libraries. You are not allowed to use NLTK, scikit-learn, or any other machine learning toolkit. You can only use scikit-learn in part 2.

### Part 1: Implementing KNN classifier from scratch (75 marks)

You are not allowed to use scikit-learn or any other machine learning toolkit for this part. You have to implement your own KNN classifier from scratch. You may use Pandas, NumPy, Matplotlib, and other standard Python libraries.

#### Problem:
The purpose of this assignment is to get you familiar with the k nearest neighbor classification. You are given the ‘Apple Sentiment Tweets’ dataset that contains around 1630 tweets about Apple labeled as positive, neutral and negative in the form of 1, 0, and -1 respectively. Your task is to implement the k nearest classifier and use it for predicting the sentiments of the tweets about Apple.


In [20]:
## Here are the libraries you will need for this part/
import pandas as pd
import numpy as np
import scipy.spatial as sc
import matplotlib.pyplot as plt
import re
import random
%matplotlib inline

#### Task 1.1: Dataset (5 points)
The dataset contains around 1,630 tweets. There are only two columns in the dataset:
Text: Contains the text of the tweet
Sentiment: Contains the sentiment of the tweet which is divided into three classes: 1 (positive), -1 (negative), and 0 (neutral).

Your task is to read the dataset and stopwords file into a useful data structure. Print out a few tweets and a few items from the stop word list, succesfully being able to do this will earn you 5 points.

In [21]:
df = pd.read_csv('/content/Apple Sentiment Tweets.csv')
df.head()

,text,sentiment
0,Wow. Yall needa step it up @Apple RT @heynyla:...,-1
1,What Happened To Apple Inc? http://t.co/FJEX...,0
2,Thank u @apple I can now compile all of the pi...,1
3,The oddly uplifting story of the Apple co-foun...,0
4,@apple can i exchange my iphone for a differen...,0


from matplotlib import pyplot as plt
_df_0['sentiment'].plot(kind='hist', bins=20, title='sentiment')
plt.gca().spines[['top', 'right',]].set_visible(False)

) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('text').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['sentiment'].plot(kind='line', figsize=(8, 4), title='sentiment')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

) missing from font(s) DejaVu Sans.
  plt.savefig(


from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3['text'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x='sentiment', y='text', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

#### Task 1.2: Data Preprocessing (10 points)

In the preprocessing step, you’re required to remove the stop words, punctuation marks, numbers, unwanted symbols, hyperlinks, and usernames from the tweets and convert them to lower case. You may find the string and regex module useful for this purpose. Use the stop word list provided within the assignment.

Print out a few random tweets from your dataset, if they conform to the rules mentioned above, you will gain 10 points.

In [22]:
import re
#removing stop words
#punctuations + numbers + unwanted symbols + hyperlinks and usernames
#convert to lower case
# Load stop words from the provided file
with open('stop_words.txt','r') as f:
  stop_words = set(f.read().splitlines())
def preprocess_tweet(text):
  text = str(text).lower()
  text = re.sub(r"http\S+|www\S+","",text)
  text = re.sub(r"@\w+","",text)
  text = re.sub(r"[^a-z\s]","",text) #remove punctuation, numbers, symbols (basically removing everything except letters)
  tokens = text.split()
  tokens = [word for word in tokens if word not in stop_words]
  return " ".join(tokens)

# Apply the preprocessing to each tweet
df['preprocessed_text'] = df['text'].apply(preprocess_tweet)

# Display a few examples to check the output
df[['text', 'preprocessed_text']].sample(5, random_state=42)

,text,preprocessed_text
670,Strategic steps towards Full-Time #Trading htt...,strategic steps towards fulltime trading stock...
251,RT @HorseshoeBmore #Holiday #Giveaway! To ensu...,rt holiday giveaway ensure stay touch go ck br...
1225,Elgato Launches Thunderbolt 2 Dock with 4K Res...,elgato launches thunderbolt dock k resolution ...
300,"YES. RT @garrett_wollman: No, @Apple, don't '...",yes rt dont remind tomorrow fing tell updates ...
352,UBS Says Consensus on December Quarter Apple ...,ubs says consensus december quarter apple ipho...


#### Task 1.3: Splitting the dataset (5 points)

In this part, divide the given dataset into training and testing sets based on an 80-20 split using python.
Print out the sizes of the training dataset and test dataset, training data should contain 1304 tweets and test data should contain 326 tweets. If your sizes are correct, you get full points.

In [23]:


from sklearn.model_selection import train_test_split

# Use the cleaned tweets and sentiments
X = df['preprocessed_text']
y = df['sentiment']

# Perform the 80-20 train-test split
#the stratify is used in order to keep the test and training data set balanced
#random_state makes it reproducable or else we will have different results each time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Print the sizes of each set
len(X_train), len(X_test)


(1304, 326)

#### Task 1.4: Feature Extraction (10 points)

In the feature extraction step, you’ll represent each tweet as a bag-of-words (BoW), that is, an unordered set of words with their position ignored, keeping only their frequency in the tweet. For example, consider the below tweets:
T1 = Welcome to machine learning!
T2 = kNN is a powerful machine learning algorithm.
The bag-of-words representation (ignoring numbers, case, and punctuation) for the above tweets are:

| Vocabulary | welcome | To | machine | learning | knn | is | a | powerful | algorithm |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| T1 | 1 | 1 | 1 | 1 | 0 | 0 | 0 | 0 | 0 |
| T2 | 0 | 0 | 1 | 1 | 1 | 1 | 1 | 1 | 1 |


Note: We only use the training set to construct the vocabulary for the BoW representation.

Print out the vocabulary as well as the bow representation for a random tweet. Getting the correct output will result in full credit.

In [24]:
import numpy as np
import re

# Simple text cleaner
def clean_text(text, stopwords):
    text = text.lower()
    text = re.sub(r"http\S+|@\S+|[^a-z\s]", "", text)
    words = text.split()
    return [word for word in words if word not in stopwords]

# 1. Clean all tweets
cleaned_texts = [clean_text(t, stop_words) for t in df["text"]]

# 2. Create vocabulary
vocab = sorted(set(word for tweet in cleaned_texts for word in tweet))
word_to_index = {word: idx for idx, word in enumerate(vocab)}

# 3. Convert to Bag-of-Words vectors
def vectorize_bow(text):
    vec = np.zeros(len(vocab))
    for word in text:
        if word in word_to_index:
            vec[word_to_index[word]] += 1
    return vec

vectors = np.array([vectorize_bow(tweet) for tweet in cleaned_texts])
labels = df["sentiment"].values


In [25]:
from sklearn.model_selection import train_test_split  # okay to use

X_train_vectors, X_test_vectors, y_train, y_test = train_test_split(
    vectors, labels, test_size=0.2, random_state=42, stratify=labels
)


#### Task 1.5: Create KNN classifier (10 points)

You will create your own k-Nearest Neighbors classifier function by performing the following tasks:
- For a test data point, find its distance from all training instances.
- Sort the calculated distances in ascending order based on distance values.
- Choose k training samples with minimum distances from the test data point.
- Return the most frequent class of these samples. (Your function should work with Euclidean distance as well as Manhattan distance. Pass the distance metric as a parameter in the KNN classifier function. Your function should also be general enough to work with any value of k.)
- For the even values of k given in the above task, break ties by backing off to the k-1 value. (For example, if you have k = 6 nearest neighbors and three of them have the label ‘positive’ and three have the label ‘negative, then you will break off the tie by taking k=5 nearest neighbors. On the other hand, let's say if you have k = 6 nearest neighbors where two have the label ‘positive’, two have the label ‘negative’, and two have the label ‘neutral’. In that case, k =5 will still lead to two labels having a draw in which case you will continue decreasing k until there is a clear winner.)


In [26]:
import numpy as np
from collections import Counter

def compute_distance(x1, x2, metric='euclidean'):
    if metric == 'euclidean':
        return np.sqrt(np.sum((x1 - x2) ** 2))
    elif metric == 'manhattan':
        return np.sum(np.abs(x1 - x2))
    else:
        raise ValueError("Unsupported distance metric")

def knn_predict(X_train, y_train, x_test, k=3, metric='euclidean'):
    distances = []

    # Compute distances
    for i in range(len(X_train)):
        dist = compute_distance(X_train[i], x_test, metric)
        distances.append((dist, y_train[i]))

    # Sort by distance
    distances.sort(key=lambda x: x[0])

    # Tie-breaking logic
    current_k = k
    while current_k > 0:
        top_k_labels = [label for _, label in distances[:current_k]]
        label_count = Counter(top_k_labels)
        most_common = label_count.most_common()

        if len(most_common) == 1 or most_common[0][1] != most_common[1][1]:
            return most_common[0][0]  # clear winner
        current_k -= 1

    return most_common[0][0]  # fallback (shouldn't happen with reasonable k)


#### Task 1.6: Implement evaluation functions (10 points)

Implement evaluation functions that calculates the:
- classification accuracy,
- F1 score,
- and the confusion matrix
of your classifier on the test set.


In [27]:
import numpy as np

def compute_accuracy(y_true, y_pred):
    correct = sum(1 for yt, yp in zip(y_true, y_pred) if yt == yp)
    return correct / len(y_true)

def compute_confusion_matrix(y_true, y_pred, labels=[-1, 0, 1]):
    label_to_index = {label: idx for idx, label in enumerate(labels)}
    matrix = np.zeros((len(labels), len(labels)), dtype=int)

    for actual, pred in zip(y_true, y_pred):
        i = label_to_index[actual]
        j = label_to_index[pred]
        matrix[i][j] += 1
    return matrix

def compute_f1_score(y_true, y_pred, labels=[-1, 0, 1]):
    matrix = compute_confusion_matrix(y_true, y_pred, labels)
    f1_scores = []

    for i, label in enumerate(labels):
        TP = matrix[i][i]
        FP = sum(matrix[j][i] for j in range(len(labels)) if j != i)
        FN = sum(matrix[i][j] for j in range(len(labels)) if j != i)

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0
        recall = TP / (TP + FN) if (TP + FN) > 0 else 0

        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        f1_scores.append(f1)

    return np.mean(f1_scores)  # macro-F1


#### Task 1.7: Cross Validation (15 points)

Use 5- fold cross-validation on your training data. (In cross-validation, you divide the training data set into 5 parts. 4 parts will be used for training and 1 part will be used for validation. Then you will take a different part of your data as a validation data set and train your algorithm on the rest of the data set.) Run your KNN function for this data for the values of k = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10. Do this for both the Euclidean distance and the Manhattan distance for each value of k.

Run your evaluation function for each value of k for both distance metrics, Report classification accuracy, F1 score, and confusion matrix.

Present the results as a graph with k values on the x-axis and classification accuracy on the y-axis. Use a single plot to compare the two versions of the classifier (one using Euclidean and the other using Manhattan distance metric). Make another graph but with the F1 score on the y-axis this time. The graphs should be properly labelled.

In [28]:
import numpy as np

def cross_validate_knn(X, y, k_neighbors=3, distance_metric='euclidean', folds=5):
    indices = np.arange(len(X))
    np.random.shuffle(indices)

    X = np.array(X)
    y = np.array(y)

    fold_size = len(X) // folds
    accuracies = []
    f1_scores = []

    for i in range(folds):
        # Split into validation and training
        start = i * fold_size
        end = (i + 1) * fold_size if i < folds - 1 else len(X)
        val_indices = indices[start:end]
        train_indices = np.concatenate((indices[:start], indices[end:]))

        X_train, y_train = X[train_indices], y[train_indices]
        X_val, y_val = X[val_indices], y[val_indices]

        y_pred = [knn_predict(X_train, y_train, x, k=k_neighbors, metric=distance_metric) for x in X_val]

        acc = compute_accuracy(y_val, y_pred)
        f1 = compute_f1_score(y_val, y_pred)
        accuracies.append(acc)
        f1_scores.append(f1)

    return np.mean(accuracies), np.mean(f1_scores)


#### Task 1.8: Classification (10 points)

Finally, use the best value of k for both distance metrics and run it on the test data set. Find the F1 score, classification accuracy, and confusion matrix and print them.

You accuracy should be above 75 and f1 score should be above 60 to get full points.

In [30]:
# --- 1. choose the search space for k ---
k_grid = [1, 3, 5, 7]   # just 4 values for speed (instead of 8+)


best_k = {}                             # to store best k per distance metric
cv_results = {}

for metric in ['euclidean', 'manhattan']:
    scores = []
    print(f"\nValidating for distance: {metric}")
    for k in k_grid:
        print(f"  → trying k = {k}")
        acc, f1 = cross_validate_knn(
            X_train_vectors, y_train,
            k_neighbors=k,
            distance_metric=metric,
            folds=3
        )
        scores.append((k, acc, f1))


    # pick k with highest F1 (break ties with higher accuracy, then lower k)
    scores.sort(key=lambda t: (-t[2], -t[1], t[0]))
    best_k[metric] = scores[0][0]
    cv_results[metric] = scores

print("Best k values:", best_k)



Validating for distance: euclidean
  → trying k = 1
  → trying k = 3
  → trying k = 5
  → trying k = 7

Validating for distance: manhattan
  → trying k = 1
  → trying k = 3
  → trying k = 5
  → trying k = 7
Best k values: {'euclidean': 1, 'manhattan': 3}


In [31]:
# --- 2. train (which for KNN just means “store” X_train & y_train) ---
def predict_batch(X_train, y_train, X_test, k, metric):
    return [
        knn_predict(X_train, y_train, x, k=k, metric=metric)
        for x in X_test
    ]

test_metrics = {}

for metric in ['euclidean', 'manhattan']:
    k_star = best_k[metric]
    y_pred = predict_batch(X_train_vectors, y_train,
                           X_test_vectors, k=k_star, metric=metric)

    acc = compute_accuracy(y_test, y_pred)
    f1  = compute_f1_score(y_test, y_pred)
    cm  = compute_confusion_matrix(y_test, y_pred)

    test_metrics[metric] = (acc, f1, cm)

    print(f"\n=== {metric.capitalize()} distance, k={k_star} ===")
    print(f"Accuracy     : {acc*100:.2f}%")
    print(f"Macro F1‑score: {f1*100:.2f}%")
    print("Confusion matrix (rows = actual, cols = predicted):\n", cm)



=== Euclidean distance, k=1 ===
Accuracy     : 66.26%
Macro F1‑score: 55.75%
Confusion matrix (rows = actual, cols = predicted):
 [[ 55  81   1]
 [  6 154   0]
 [  0  22   7]]

=== Manhattan distance, k=3 ===
Accuracy     : 60.12%
Macro F1‑score: 44.78%
Confusion matrix (rows = actual, cols = predicted):
 [[ 34 103   0]
 [  2 158   0]
 [  1  24   4]]


### Part 2:  kNN classifier using scikit-learn (25 points)

In this part, you have to use scikit-learn’s KNN implementation to train and test your classifier on the dataset used in Part 1. Repeat the tasks you have done in Part 1 but this time using scikit-learn. Use your bag of words to do cross-validation and run the KNN classifier for values of k = 1, 2, 3, 4, 5, 6, 7, 8, 9, 10 using both Euclidean and Manhattan distance. Use scikit-learn’s accuracy score function to calculate the accuracy, classification_report to calculate macro-average (precision, recall, and F1), and confusion matrix function to calculate confusion matrix for test data. Also present the results as a graph with k values on the x-axis and performance measures on the y-axis just like you did in part 1. Use a single plot to compare the two versions of the classifier (one using Euclidean and the other using Manhattan distance metric). Finally, print the best values of k for both distance metrics. Then use this value of k on the test data set and print the evaluation scores.

To get full marks, the accuracy score, classification reports and confusion matrix must be shown for both distance metrics and values for accuracy and F1 should be similar to those obtained in the previous part.

In [32]:
# Here are the libraries and specific functions you will be needing for this part

from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_predict

In [33]:
# ✅ 1. Imports
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
import pandas as pd

# ✅ 2. Basic text cleaner (in case you haven’t already cleaned the tweets)
import re
def clean_text(text, stopwords):
    text = text.lower()
    text = re.sub(r"http\S+|@\S+|[^a-z\s]", "", text)
    words = text.split()
    return " ".join([word for word in words if word not in stopwords])

# Apply cleaning
df["text_cleaned"] = df["text"].apply(lambda x: clean_text(x, stop_words))

# ✅ 3. Vectorization (you can switch to TfidfVectorizer if needed)
vectorizer = CountVectorizer()
X_vectors = vectorizer.fit_transform(df["text_cleaned"])
y = df["sentiment"].values

# ✅ 4. Train/test split
X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ 5. Cross-validation to find best k
print("🔎 Cross-validating to find best k...")
best_k = None
best_score = 0
for k in [3, 5, 7, 9]:
    clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')
    print(f"k = {k}: mean F1-macro = {scores.mean():.4f}")
    if scores.mean() > best_score:
        best_score = scores.mean()
        best_k = k

# ✅ 6. Final model training + test evaluation
print(f"\n🎯 Best k = {best_k}, training final model...")

final_clf = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
final_clf.fit(X_train, y_train)
y_pred = final_clf.predict(X_test)

acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
cm = confusion_matrix(y_test, y_pred)

# ✅ 7. Results
print("\n📊 Final Evaluation on Test Set:")
print(f"Accuracy     : {acc*100:.2f}%")
print(f"Macro F1‑Score: {f1*100:.2f}%")
print("Confusion Matrix:\n", cm)


🔎 Cross-validating to find best k...
k = 3: mean F1-macro = 0.4499
k = 5: mean F1-macro = 0.4423
k = 7: mean F1-macro = 0.4367
k = 9: mean F1-macro = 0.4461

🎯 Best k = 3, training final model...

📊 Final Evaluation on Test Set:
Accuracy     : 64.42%
Macro F1‑Score: 51.48%
Confusion Matrix:
 [[ 51  86   0]
 [  6 154   0]
 [  2  22   5]]


Vectorize the tweets

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(stop_words=stop_words)  # you already loaded stop_words earlier
X_vectors = vectorizer.fit_transform(df["text_cleaned"])  # assuming you already cleaned tweets
y = df["sentiment"].values


Split into train and test

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_vectors, y, test_size=0.2, random_state=42, stratify=y
)


Train with KNN Classifier + Cross-Validation

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

# Try different k values
for k in [3, 5, 7, 9]:
    clf = KNeighborsClassifier(n_neighbors=k, metric='euclidean')
    scores = cross_val_score(clf, X_train, y_train, cv=5, scoring='f1_macro')
    print(f"k={k}, mean F1-macro: {scores.mean():.4f}")


Final Evaluation on test set

In [ ]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Train best model
best_k = 5  # change based on previous step
clf = KNeighborsClassifier(n_neighbors=best_k, metric='euclidean')
clf.fit(X_train, y_train)

# Predict and evaluate
y_pred = clf.predict(X_test)

print("Accuracy     :", accuracy_score(y_test, y_pred))
print("F1 Macro     :", f1_score(y_test, y_pred, average='macro'))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
